In [ ]:
from agential.cog.prompts.critic import (
    HUMANEVAL_FEWSHOT_EXAMPLES_POT, 
    CRITIC_POT_INSTRUCTION_HUMANEVAL,
    CRITIC_POT_INSTRUCTION_TEST_HUMANEVAL,
    HUMANEVAL_FEWSHOT_EXAMPLES_POT_TEST,
    CRITIC_CRITIQUE_INSTRUCTION_HUMANEVAL,
    HUMANEVAL_FEWSHOT_EXAMPLES_CRITIC
)
from agential.cog.functional.critic import _prompt_agent, safe_execute, _prompt_critique

import dotenv
import warnings
warnings.filterwarnings("ignore")

from langchain_community.chat_models.openai import ChatOpenAI

llm = ChatOpenAI()

In [ ]:
question = """from typing import List 

def has_close_elements(numbers: List[float], threshold: float) -> bool: 
    \"\"\"Check if in given list of numbers, are any two numbers closer to each other than given threshold. 
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5) False 
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3) True
    \"\"\"
"""

code = _prompt_agent(
    llm=llm,
    question=question,
    examples=HUMANEVAL_FEWSHOT_EXAMPLES_POT,
    prompt=CRITIC_POT_INSTRUCTION_HUMANEVAL,
)

In [ ]:
print(code)

In [ ]:
tests = _prompt_agent(
    llm=llm,
    question=question,
    examples=HUMANEVAL_FEWSHOT_EXAMPLES_POT_TEST,
    prompt=CRITIC_POT_INSTRUCTION_TEST_HUMANEVAL
)

In [ ]:
print(tests) 

In [ ]:
code_answer, execution_status = safe_execute(
    code + "\n\n" + tests #+ "assert False" + "\n\n"
)  # Can be None, "Exception".

In [ ]:
execution_status

In [13]:
a = """
def are_anagrams(s1: str, s2: str) -> bool:
    \"\"\"Check if two strings are anagrams of each other, ignoring case.
    >>> are_anagrams('Listen', 'Silent')
    True
    >>> are_anagrams('Hello', 'World')
    False
    >>> are_anagrams('Angel', 'Glean')
    True
    \"\"\"
    from collections import Counter
    return Counter(s1) == Counter(s2)

assert are_anagrams('Listen', 'Silent') == True, "Test failed: are_anagrams('Listen', 'Silent') should return True"
assert are_anagrams('Hello', 'World') == False, "Test failed: are_anagrams('Hello', 'World') should return False"
assert are_anagrams('Angel', 'Glean') == True, "Test failed: are_anagrams('Angel', 'Glean') should return True"
"""

safe_execute(a)

In [ ]:
critique_additional_keys = {
    "execution_status": execution_status,
    "code_answer": code_answer,
}

critique = _prompt_critique(
    llm=llm,
    question=code,
    examples=critique_examples,
    answer=tests,
    critique="",
    additional_keys=critique_additional_keys,
    prompt=critique_prompt
)